In [ ]:
#You might need to install obspy if you don't already have it (written with ObsPy 1.0.1)
#%sx pip install obspy

In [ ]:
import obspy
import requests
import json

from obspy import read
import matplotlib.pyplot as plt

Function to convert a Stream to a (minimal) Waveform object

In [ ]:
def irisTraceToWaveformJson(stream, evid, originTime, evLat, evLon):   
    obj = []
    for tr in stream.traces:        
        stat = tr.stats
        inventory = client.get_stations(network=stat.network, station=stat.station,
                                starttime=stat.starttime,
                                endtime=stat.endtime)  
        wave = {}
        event = {}
        event["eventId"] = evid
        event["originTime"] = originTime
        event["latitude"] = evLat
        event["longitude"] = evLon
        wave["event"] = event
        
        stream = {}
        station = {}
        station["networkName"] = stat.network
        station["stationName"] = stat.station
        station["latitude"] = inventory.networks[0].stations[0].latitude
        station["longitude"] = inventory.networks[0].stations[0].longitude
        stream["station"] = station;
        stream["channelName"] = stat.channel
        wave["stream"] = stream
        
        wave["beginTime"] = str(stat.starttime)
        wave["endTime"] = str(stat.endtime)
        wave["sampleRate"] = stat.sampling_rate
        wave["segment"] = tr.data.tolist()
        obj.append(wave)
    
    envelopeJob = {}
    envelopeJob["data"] = obj
    
    return envelopeJob

### Upload and run the calibration and envelope application services from the terminal before the rest of the notebook

java -jar envelope-application-1.0.*-runnable.jar

java -jar calibration-application-1.0.*-runnable.jar

Request data from NCEDC

In [ ]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
#We could get data from IRIS but we just want Northern Cal data since thats our calibrated model's region
client = Client("http://service.ncedc.org/")

Here I am just picking an event that I know the calibration should be valid for given the stations in question

In [ ]:
starttime = UTCDateTime("2007/10/31")
endtime = UTCDateTime("2007/11/01")
cat = client.get_events(starttime=starttime, endtime=endtime,
                        minmagnitude=4.0, minlatitude=36.5, maxlatitude=38.5, minlongitude=-123, maxlongitude=-121)
print(cat)

In [ ]:
st = None

for event in cat.events:
    t = event.origins[0].time
    evid = str(event.resource_id).split("/")[3]
    evlat = event.origins[0].latitude
    evlon = event.origins[0].longitude
    stations = ["BRK", "BKS", "CVS", "MHC", "SAO", "WENL"]
    for sta in stations :
        ste = client.get_waveforms("BK", sta, "*", "BHE", t - 500, t + 3000, attach_response=True)
        stn = client.get_waveforms("BK", sta, "*", "BHN", t - 500, t + 3000, attach_response=True)

        #Instrument correction may be required
        pre_filt = (0.005, 0.006, 30.0, 35.0)           
        ste.remove_response(output='VEL', pre_filt=pre_filt)
        stn.remove_response(output='VEL', pre_filt=pre_filt)

        #ObsPy defaults to m/s for VEL and we want NM/s
        for trace in ste.traces:
            trace.data = trace.data * 1e+9
        for trace in stn.traces:
            trace.data = trace.data * 1e+9        
                    
        if (st == None):
            st = ste + stn
        else:
            st = st + ste + stn    

print(st)

Convert the ObsPy Stream into Waveform JSON

In [ ]:
waveData = irisTraceToWaveformJson(st, evid, str(t), evlat, evlon)

The service uses self signed certs, unless you specify ones to use, so for this example we just ignore the ssl verifcation errors. 

### Do not do this if you are using anything not on localhost in this notebook!

In [ ]:
requests.packages.urllib3.disable_warnings()

Post the frequency band based model parameters

In [ ]:
with open("norcal_bands.json") as bands:
    norcal_bands = json.load(bands)
    r = requests.post('https://127.0.0.1:53921/api/v1/params/shared-fb-parameters/batch', json=norcal_bands, verify=False)
    print(r.status_code)
    print(r.text)

Post the site corrections to the service for northern california (obtained from running the calibration service, not shown here).

In [ ]:
with open("norcal_site.json") as corrections:
    norcal_corrections = json.load(corrections)
    r = requests.post('https://127.0.0.1:53921/api/v1/params/site-fb-parameters/batch', json=norcal_corrections, verify=False)
    print(r.status_code)

Post the data to the envelopes service and decode the resulting stacks. Note that I am not posting envelope bands in this request and just relying on the default configuration so this will output fewer bands than we actually calibrated against. In practice I would want to submit the bands with the job to get the full spectra I have calibrations for (see EnvelopeJob for the format for that).

In [ ]:
r = requests.post('https://127.0.0.1:53922/api/v1/envelopes/create/batch-stacks-only/10000', json=waveData, verify=False)
print(r.status_code)
envData = json.loads(r.text)["resultPayload"]

Post data to Mw endpoint and receive the measurements back

In [ ]:
request = {
    "stacks": envData,
    "autopickingEnabled":"true"
}
r = requests.post('https://127.0.0.1:53921/api/v1/measurement/measure-mws', json=request, verify=False, timeout=10000)
print(r.status_code)
eventMws = json.loads(r.text)

In [ ]:
print(eventMws["measuredMwDetails"]["40204628"])

Plot the returned Mws vs the NCEDC Mws

In [ ]:
fig, ax = plt.subplots()
x1 = [cat.events[0].magnitudes[0].mag]
x2 = []
for evs in eventMws["measuredMwDetails"]:
    x2.append(eventMws["measuredMwDetails"][evs]["mw"])    
scale = 200.0

ax.scatter(x1,x1, c='blue', s=scale, label='NC Mw',
           alpha=1, edgecolors='none')

ax.scatter(x2,x2, c='red', s=scale, label='CCT Mw',
           alpha=1, edgecolors='none')

ax.legend()
ax.grid(True)

plt.show()